# Cluster analysis example

### Importing MLlib libraries 

In [305]:
import pyspark.mllib.clustering
from pyspark.mllib.clustering import KMeans
import pyspark.mllib.linalg
from pyspark.mllib.linalg import Vectors
from pyspark import SparkConf, SparkContext

### Read data and pre-processing

In [ ]:
conf = SparkConf()
sc = SparkContext(conf=conf)

In [254]:
rawData = sc.textFile("data/iris.csv")

The following code splits the CSV lines into columns and removes the final column. The remaining values
are converted to an array of numeric values (Double objects), and emitted with the final label column in a
tuple.

In [6]:
#rawData.map(_.split(',').last).countByValue().toSeq.sortBy(_._2).reverse.foreach(println)
for x in sorted(rawData.map(lambda x:x.split(',')[-1]).countByValue().items(), key = lambda data:data[1], reverse=True):
    print(x)
'''
val labelsAndData = rawData.map { line =>
    val buffer = line.split(',').toBuffer
    val label = buffer.remove(buffer.length - 1)
    val vector = Vectors.dense(buffer.map(_.toDouble).toArray)
    (label, vector)
}

val data = labelsAndData.values.cache()

'''

def estractLabelVector(line):
    buff = line.split(',')
    label = buff[-1]
    vector = Vectors.dense([float(c) for c in buff[:-1]])
    return (label, vector)

labelsAndData = rawData.map(estractLabelVector)

data = labelsAndData.values().cache()


('Iris-setosa', 50)
('Iris-virginica', 50)
('Iris-versicolor', 50)


### Perform cluster analysis

In [7]:
'''
val numClusters = 2
val numIterations = 1
val model = KMeans.train(data,numClusters,numIterations)
'''
numClusters = 2
numIterations = 1
model = KMeans.train(data,numClusters,numIterations)

In [8]:
#model.clusterCenters.foreach(println)
for x in model.clusterCenters:
    print(x)

[ 5.00566038  3.36981132  1.56037736  0.29056604]
[ 6.30103093  2.88659794  4.95876289  1.69587629]


### Apply cluster model
The following code uses the model to assign each observation to a cluster, counts occurrences of cluster and
label pairs, and prints them.

In [9]:
'''
val clusterLabelCount = labelsAndData.map { case (label, datum) =>
  val cluster = model.predict(datum)
  (cluster, label)}.countByValue()
  clusterLabelCount.toSeq.sorted.foreach { case ((cluster, label), count) =>
  println(f"$cluster%1s$label%18s$count%8s")
  }
'''

def match(data):
    cluster = model.predict(data[1])
    return (cluster, data[0])

clusterLabelCount = labelsAndData.map(match).countByValue()

for key in clusterLabelCount.keys():
    print("%1s%18s%8s" % (key[0], key[1], clusterLabelCount[key]))


0       Iris-setosa      50
1    Iris-virginica      50
1   Iris-versicolor      47
0   Iris-versicolor       3


### Write sample and total result in a directory hadoop style and coalesce all in one file

In [10]:
'''  
val sample = data.map(datum => model.predict(datum) + "," +datum.toArray.mkString(",")).sample(false,0.05)
val total = data.map(datum => model.predict(datum) + "," +datum.toArray.mkString(","))
sample.saveAsTextFile("share/sample")
sample.coalesce(1).saveAsTextFile("share/sample_coalesce")
'''
total = data.map(lambda datum: str(model.predict(datum)) + "," +",".join([str(c) for c in datum]))
sample = total.sample(False,0.05)
sample.saveAsTextFile("share/sample")
sample.coalesce(1).saveAsTextFile("share/sample_coalesce")

Py4JJavaError: An error occurred while calling o58.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/home/jovyan/work/share/sample already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:132)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1089)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1065)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1065)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:147)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:108)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:286)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1065)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$4.apply$mc

### Choice of K
A clustering could be considered good if each data point were near to its closest centroid. So, we define a
Euclidean distance function, and a function that returns the distance from a data point to its nearest cluster’s
centroid. From this, it’s possible to define a function that measures the average distance to centroid, for a
model built with a given k. This is an internal quality measure.

In [98]:
#def distance(a: Vector, b: Vector) =math.sqrt(a.toArray.zip(b.toArray).map(p => p._1 - p._2).map(d => d * d).sum)
import numpy as np

def distance(a, b):
    return np.linalg.norm(np.asarray(a)-np.asarray(b))

'''
def distToCentroid(datum: Vector, model: KMeansModel) = {
      val cluster = model.predict(datum)
      val centroid = model.clusterCenters(cluster)
      distance(centroid, datum)
}   
'''
def distToCentroid(datum, model):
    cluster = model.predict(datum)
    centroid = model.clusterCenters[cluster]
    return distance(centroid, datum)

'''
def clusteringScore(data: RDD[Vector], k: Int): Double = {
      val kmeans = new KMeans()
      kmeans.setK(k)
      kmeans.setRuns(30)
      kmeans.setEpsilon(1.0e-6)
      val model = kmeans.run(data)
      data.map(datum => distToCentroid(datum, model)).mean()
}
'''
def clusteringScore(data, k):
    model = KMeans.train(data, k, runs=30, epsilon=1.0e-6)
    return data.map(lambda datum: distToCentroid(datum, model)).mean()

Evaluate score values for different k from 1 to 10.

In [109]:
#(1 to 10 by 1).map(k => (k, clusteringScore(data, k))).foreach(println)
for k in range(10):
    print((k+1,clusteringScore(data,k+1)))

(1, 1.9440683605553895)
(2, 0.85557776952665299)
(3, 0.64803049049344341)
(4, 0.55738477273333131)
(5, 0.50965219511182869)
(6, 0.4670733963396439)
(7, 0.43616625646364982)
(8, 0.41086768314893807)
(9, 0.39617960125111995)
(10, 0.38159011216911809)


### Data Normalization
Since Euclidean distance is used, the clusters will be influenced strongly by the magnitudes of the variables, especially by outliers. Normalizing removes this bias. We can normalize each feature by converting it to a standard score. This means subtracting the mean of the feature’s values from each value, and dividing by the standard deviation.

In [238]:
'''
def buildNormalizationFunction(data: RDD[Vector]): (Vector => Vector) = {
    val dataAsArray = data.map(_.toArray)
    val numCols = dataAsArray.first().length
    val n = dataAsArray.count()
    val sums = dataAsArray.reduce(
      (a, b) => a.zip(b).map(t => t._1 + t._2))
    val sumSquares = dataAsArray.fold(
        new Array[Double](numCols)
      )(
        (a, b) => a.zip(b).map(t => t._1 + t._2 * t._2)
      )
    val stdevs = sumSquares.zip(sums).map {
      case (sumSq, sum) => math.sqrt(n * sumSq - sum * sum) / n
    }
    val means = sums.map(_ / n)

    (datum: Vector) => {
      val normalizedArray = (datum.toArray, means, stdevs).zipped.map(
        (value, mean, stdev) =>
          if (stdev <= 0)  (value - mean) else  (value - mean) / stdev
      )
      Vectors.dense(normalizedArray)
    }
}
'''

def buildNormalizationFunction(data):
    dataAsArray = data.map(lambda datum: np.asarray(datum))
    numCols = dataAsArray.first().size
    n = dataAsArray.count()
    sums = dataAsArray.reduce(lambda a,b: [t1 + t2 for t1, t2 in zip(a,b)])
    sumSquares = dataAsArray.reduce(lambda a,b: [t1 + t2*t2 for t1, t2 in zip(a,b)])
    stdevs = [(n * sumSq - sum**2)**(1/2)/n for sumSq, sum in zip(sumSquares, sums)]
    means = [s/n for s in sums]
    return dataAsArray.map(lambda datum :
                           Vectors.dense([value - mean if stdev == 0 else (value - mean) / stdev
                                          for value, mean, stdev in zip(datum, means, stdevs)])
                          )

In [242]:
normalizedData = buildNormalizationFunction(data).cache()

In [243]:
'''
(1 to 10 by 1).map(k =>
      (k, clusteringScore(normalizedData, k))).foreach(println)
'''
for k in range(10):
    print((k+1,clusteringScore(normalizedData,k+1)))

(1, 0.036432654871608426)
(2, 0.01634733527199109)
(3, 0.010421227668008742)
(4, 0.0087876646616176085)
(5, 0.0078241687917766645)
(6, 0.0071239033848412171)
(7, 0.0066003290868659964)
(8, 0.0061642170075782831)
(9, 0.0057914506083838859)
(10, 0.0056075297544857438)


### Definition of Entropy measure
The are different metrics for homogeneity. Entropy is used here for illustration. A good clustering would have clusters whose collections of labels are homogeneous and so have low entropy. A weighted average of entropy can therefore be used as a cluster score. This is an external quality measure.

In [248]:
'''
def entropy(counts: Iterable[Int]) = {
    val values = counts.filter(_ > 0)
    val n: Double = values.sum
    values.map { v =>
    val p = v / n 
    -p * math.log(p)
    }.sum
}
'''
import math
def entropy(counts):
    values = filter(lambda x: x>0, counts)
    n = float(sum(values))
    return sum(map(lambda v: - v/n * math.log(v/n), values))

In [257]:
'''
def buildCategoricalAndLabelFunction(rawData: RDD[String]): (String => (String,Vector)) = {
val splitData = rawData.map(_.split(','))
   (line: String) => {
      val buffer = line.split(',').toBuffer 
      val label = buffer.remove(buffer.length - 1)
      val vector = buffer.map(_.toDouble)
      (label, Vectors.dense(vector.toArray))
    }
}
'''
def buildCategoricalAndLabelFunction(rawData):
    return rawData.map(estractLabelVector)

### Entropy for choosing K

In [304]:
'''
def clusteringScore3(normalizedLabelsAndData: RDD[(String,Vector)], k: Int) = {
    val kmeans = new KMeans()
    kmeans.setK(k)
    kmeans.setRuns(10)
    kmeans.setEpsilon(1.0e-6)
    val model = kmeans.run(normalizedLabelsAndData.values)
    val labelsAndClusters = normalizedLabelsAndData.mapValues(model.predict)
    val clustersAndLabels = labelsAndClusters.map(_.swap)
    val labelsInCluster = clustersAndLabels.groupByKey().values
    val labelCounts = labelsInCluster.map(_.groupBy(l => l).map(_._2.size))
    val n = normalizedLabelsAndData.count()
    labelCounts.map(m => m.sum * entropy(m)).sum / n
}

val parseFunction = buildCategoricalAndLabelFunction(rawData)
val labelsAndData = rawData.map(parseFunction)
val normalizedLabelsAndData =
      labelsAndData.mapValues(buildNormalizationFunction(labelsAndData.values)).cache()
(1 to 10 by 1).map(k =>
      (k, clusteringScore3(normalizedLabelsAndData, k))).foreach(println)
normalizedLabelsAndData.unpersist()
'''
from itertools import groupby
def clusteringScore3(normalizedLabelsAndData, k):
    model = KMeans.train(normalizedLabelsAndData.values(), k, runs=30, epsilon=1.0e-6)
    labelsAndClusters = normalizedLabelsAndData.mapValues(model.predict)
    clustersAndLabels = labelsAndClusters.map(lambda x: (x[1],x[0]))
    labelsInCluster = clustersAndLabels.groupByKey().values()
    labelCounts = labelsInCluster.map(lambda x: [len(list(group)) for key, group in groupby(x, lambda l:l)])
    n = float(normalizedLabelsAndData.count())
    return labelCounts.map(lambda m: sum(m) * entropy(m)).sum() / n
    
def buildNormalizationFunctionWithLabels(data):
    dataAsArray = data.values().map(lambda datum: np.asarray(datum))
    numCols = dataAsArray.first().size
    n = dataAsArray.count()
    sums = dataAsArray.reduce(lambda a,b: [t1 + t2 for t1, t2 in zip(a,b)])
    sumSquares = dataAsArray.reduce(lambda a,b: [t1 + t2*t2 for t1, t2 in zip(a,b)])
    stdevs = [(n * sumSq - sum**2)**(1/2)/n for sumSq, sum in zip(sumSquares, sums)]
    means = [s/n for s in sums]
    return data.map(lambda datum :
                           (datum[0], Vectors.dense([value - mean if stdev == 0 else (value - mean) / stdev
                                                     for value, mean, stdev in zip(datum[1], means, stdevs)]))
                          )
    
labelsAndData = buildCategoricalAndLabelFunction(rawData)
normalizedLabelsAndData = buildNormalizationFunctionWithLabels(labelsAndData)
for k in range(10):
    print((k+1,clusteringScore3(normalizedLabelsAndData,k+1)))
normalizedLabelsAndData.unpersist()

(1, 0.0)
(2, 0.0)
(3, 0.0)
(4, 0.0)
(5, 0.0)
(6, 0.0)
(7, 0.0)
(8, 0.0)
(9, 0.0)
(10, 0.0)


PythonRDD[8256] at RDD at PythonRDD.scala:43